In [1]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.datasets import load_digits
from sklearn.datasets import load_iris

set_config(print_changed_only=False)


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

%config InlineBackend.figure_format='svg'
plt.rcParams['figure.dpi']=120

pd.options.display.float_format='{:,.2f}'.format
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings('ignore')


<h1 style='color:blue' align='center'>Bagging: Ensenmbel Learning</h1>

<img src='./data/Bagging.jpg' width=1000 height=600>

<img src='./data/Bagging_1.jpg' width=600 height=400>

<a href="http://www.differencebetween.net/technology/difference-between-bagging-and-random-forest/">Source</a>

In [2]:
df = pd.read_csv("./data/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.60,0.63,50,1
1,1,85,66,29,0,26.60,0.35,31,0
2,8,183,64,0,0,23.30,0.67,32,1
3,1,89,66,23,94,28.10,0.17,21,0
4,0,137,40,35,168,43.10,2.29,33,1


In [3]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [4]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.00,768.00,768.00,768.00,768.00,768.00,768.00,768.00,768.00
mean,3.85,120.89,69.11,20.54,79.80,31.99,0.47,33.24,0.35
std,3.37,31.97,19.36,15.95,115.24,7.88,0.33,11.76,0.48
min,0.00,0.00,0.00,0.00,0.00,0.00,0.08,21.00,0.00
25%,1.00,99.00,62.00,0.00,0.00,27.30,0.24,24.00,0.00
50%,3.00,117.00,72.00,23.00,30.50,32.00,0.37,29.00,0.00
75%,6.00,140.25,80.00,32.00,127.25,36.60,0.63,41.00,1.00
max,17.00,199.00,122.00,99.00,846.00,67.10,2.42,81.00,1.00


In [5]:
df.Outcome.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

### Train test split

In [6]:
X = df.drop("Outcome",axis="columns")
y = df.Outcome

In [7]:

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:3]

array([[ 0.63994726,  0.84832379,  0.14964075,  0.90726993, -0.69289057,
         0.20401277,  0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575,  0.53090156, -0.69289057,
        -0.68442195, -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, -1.28821221, -0.69289057,
        -1.10325546,  0.60439732, -0.10558415]])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=10)

In [9]:
X_train.shape
X_test.shape

(576, 8)

(192, 8)

In [10]:
y_train.value_counts()

0    375
1    201
Name: Outcome, dtype: int64

In [11]:
201/375

0.536

In [12]:
y_test.value_counts()

0    125
1     67
Name: Outcome, dtype: int64

In [13]:
67/125

0.536

### Train using stand alone model

In [14]:
scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)
scores

scores.mean()

array([0.69480519, 0.66233766, 0.67532468, 0.79084967, 0.71895425])

0.708454290807232

### Train using Bagging

In [15]:
bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
bag_model.fit(X_train, y_train)
bag_model.oob_score_

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        random_state=None,
                                                        splitter='best'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
       

0.7534722222222222

In [16]:
bag_model.score(X_test, y_test)

0.7760416666666666

In [17]:
bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
scores = cross_val_score(bag_model, X, y, cv=5)
scores
scores.mean()

array([0.75324675, 0.72727273, 0.74675325, 0.82352941, 0.73856209])

0.7578728461081402

We can see some improvement in test score with bagging classifier as compared to a standalone classifier

### Train using Random Forest

In [18]:
scores = cross_val_score(RandomForestClassifier(n_estimators=50), X, y, cv=5)
scores.mean()

0.7630846277905101

### Exercise 

In [19]:
df = pd.read_csv("./data/heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.00,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.00,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.00,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.50,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.00,Up,0


In [20]:
df.shape
df.describe()

(918, 12)

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.00,918.00,918.00,918.00,918.00,918.00,918.00
mean,53.51,132.40,198.80,0.23,136.81,0.89,0.55
std,9.43,18.51,109.38,0.42,25.46,1.07,0.50
min,28.00,0.00,0.00,0.00,60.00,-2.60,0.00
25%,47.00,120.00,173.25,0.00,120.00,0.00,0.00
50%,54.00,130.00,223.00,0.00,138.00,0.60,1.00
75%,60.00,140.00,267.00,0.00,156.00,1.50,1.00
max,77.00,200.00,603.00,1.00,202.00,6.20,1.00


<h3>Treat Outliers</h3>

In [21]:
df[df.Cholesterol>(df.Cholesterol.mean()+3*df.Cholesterol.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
76,32,M,ASY,118,529,0,Normal,130,N,0.00,Flat,1
149,54,M,ASY,130,603,1,Normal,125,Y,1.00,Flat,1
616,67,F,NAP,115,564,0,LVH,160,N,1.60,Flat,0


In [22]:
df.shape

(918, 12)

In [23]:
df1 = df[df.Cholesterol<=(df.Cholesterol.mean()+3*df.Cholesterol.std())]
df1.shape

(915, 12)

In [24]:
df[df.MaxHR>(df.MaxHR.mean()+3*df.MaxHR.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease


In [25]:
df[df.FastingBS>(df.FastingBS.mean()+3*df.FastingBS.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease


In [26]:
df[df.Oldpeak>(df.Oldpeak.mean()+3*df.Oldpeak.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
166,50,M,ASY,140,231,0,ST,140,Y,5.00,Flat,1
702,59,M,TA,178,270,0,LVH,145,N,4.20,Down,0
771,55,M,ASY,140,217,0,Normal,111,Y,5.60,Down,1
791,51,M,ASY,140,298,0,Normal,122,Y,4.20,Flat,1
850,62,F,ASY,160,164,0,LVH,145,N,6.20,Down,1
900,58,M,ASY,114,318,0,ST,140,N,4.40,Down,1


In [27]:
df2 = df1[df1.Oldpeak<=(df1.Oldpeak.mean()+3*df1.Oldpeak.std())]
df2.shape

(909, 12)

In [28]:
df[df.RestingBP>(df.RestingBP.mean()+3*df.RestingBP.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
109,39,M,ATA,190,241,0,Normal,106,N,0.00,Up,0
241,54,M,ASY,200,198,0,Normal,142,Y,2.00,Flat,1
365,64,F,ASY,200,0,0,Normal,140,Y,1.00,Flat,1
399,61,M,NAP,200,0,1,ST,70,N,0.00,Flat,1
592,61,M,ASY,190,287,1,LVH,150,Y,2.00,Down,1
732,56,F,ASY,200,288,1,LVH,133,Y,4.00,Down,1
759,54,M,ATA,192,283,0,LVH,195,N,0.00,Up,1


In [29]:
df3 = df2[df2.RestingBP<=(df2.RestingBP.mean()+3*df2.RestingBP.std())]
df3.shape

(902, 12)

In [30]:
df.ChestPainType.unique()

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [31]:
df.RestingECG.unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [32]:
df.ExerciseAngina.unique()

array(['N', 'Y'], dtype=object)

In [33]:
df.ST_Slope.unique()

array(['Up', 'Flat', 'Down'], dtype=object)

### Handle text columns using label encoding and one hot encoding

In [34]:
df4 = df3.copy()
df4.ExerciseAngina.replace(
    {
        'N': 0,
        'Y': 1
    },
    inplace=True)

df4.ST_Slope.replace(
    {
        'Down': 1,
        'Flat': 2,
        'Up': 3
    },
    inplace=True
)

df4.RestingECG.replace(
    {
        'Normal': 1,
        'ST': 2,
        'LVH': 3
    },
    inplace=True)

df4.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,1,172,0,0.00,3,0
1,49,F,NAP,160,180,0,1,156,0,1.00,2,1
2,37,M,ATA,130,283,0,2,98,0,0.00,3,0
3,48,F,ASY,138,214,0,1,108,1,1.50,2,1
4,54,M,NAP,150,195,0,1,122,0,0.00,3,0


In [35]:
df5 = pd.get_dummies(df4, drop_first=True)
df5.head()

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,140,289,0,1,172,0,0.00,3,0,1,1,0,0
1,49,160,180,0,1,156,0,1.00,2,1,0,0,1,0
2,37,130,283,0,2,98,0,0.00,3,0,1,1,0,0
3,48,138,214,0,1,108,1,1.50,2,1,0,0,0,0
4,54,150,195,0,1,122,0,0.00,3,0,1,0,1,0


In [36]:
X = df5.drop("HeartDisease",axis='columns')
y = df5.HeartDisease

X.head()

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,140,289,0,1,172,0,0.00,3,1,1,0,0
1,49,160,180,0,1,156,0,1.00,2,0,0,1,0
2,37,130,283,0,2,98,0,0.00,3,1,1,0,0
3,48,138,214,0,1,108,1,1.50,2,0,0,0,0
4,54,150,195,0,1,122,0,0.00,3,1,0,1,0


In [37]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-1.42896269,  0.46089071,  0.85238015, ...,  2.06757196,
        -0.53547478, -0.22914788],
       [-0.47545956,  1.5925728 , -0.16132855, ..., -0.4836591 ,
         1.86750159, -0.22914788],
       [-1.74679706, -0.10495034,  0.79657967, ...,  2.06757196,
        -0.53547478, -0.22914788],
       ...,
       [ 0.37209878, -0.10495034, -0.61703246, ..., -0.4836591 ,
        -0.53547478, -0.22914788],
       [ 0.37209878, -0.10495034,  0.35947592, ...,  2.06757196,
        -0.53547478, -0.22914788],
       [-1.64085227,  0.3477225 , -0.20782894, ..., -0.4836591 ,
         1.86750159, -0.22914788]])

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=20)

In [39]:
X_train.shape
X_test.shape

(721, 13)

(181, 13)

### Train a model using standalone support vector machine and then using bagging

In [40]:
scores = cross_val_score(SVC(), X, y, cv=5)
scores.mean()

0.6906445672191528

### Use bagging now with svm

In [41]:
bag_model = BaggingClassifier(base_estimator=SVC(), n_estimators=100, max_samples=0.8, random_state=0)
scores = cross_val_score(bag_model, X, y, cv=5)
scores.mean()

0.6839656230816453

**Using bagging in case of SVM doesn't make much difference in terms of model accuracy. Bagging is effective when we have <b style='color:blue'>high variance and instable model such as decision tree.</b> Let's explore how bagging changes the performance for a decision tree classifier.**

### Train a model using decision tree and then using bagging

In [42]:
scores = cross_val_score(DecisionTreeClassifier(random_state=0), X, y, cv=5)
scores.mean()

0.7193984039287907

### Use bagging now with decision tree

In [43]:
bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(random_state=0), 
    n_estimators=100, 
    max_samples=0.9, 
    oob_score=True,
    random_state=0
)

scores = cross_val_score(bag_model, X, y, cv=5)
scores.mean()

0.8037016574585636

**With bagging the score improved from 71.93% to 80.37%**

### Train a model using Random Forest which itself uses bagging underneath

In [44]:
scores = cross_val_score(RandomForestClassifier(), X, y, cv=5)
scores.mean()

0.8203621853898098

**Random forest gave even a better performance with 82.04% as score. Underneath it used bagging where it sampled not only data rows but also the columns (or features)**